# ３章　可視化の仕組みを構築する１０本ノック

### ノック２１：店舗を絞り込んで可視化できるようにしてみよう

In [ ]:
import pandas as pd
from IPython.display import display, clear_output

m_store = pd.read_csv("m_store.csv")
m_area = pd.read_csv("m_area.csv")
order_data = pd.read_csv("tbl_order_202004.csv")
order_data = pd.merge(order_data, m_store, on="store_id", how="left")
order_data = pd.merge(order_data, m_area, on="area_cd", how="left")

# マスターにないコードに対応した文字列を設定
order_data.loc[order_data["takeout_flag"]==0, "takeout_name"] = "デリバリー"
order_data.loc[order_data["takeout_flag"]==1, "takeout_name"] = "お持ち帰り"

order_data.loc[order_data["status"]==0, "status_name"] = "受付"
order_data.loc[order_data["status"]==1, "status_name"] = "お支払済"
order_data.loc[order_data["status"]==2, "status_name"] = "お渡し済"
order_data.loc[order_data["status"]==9, "status_name"] = "キャンセル"

order_data.head()

In [ ]:
from ipywidgets import Dropdown

def order_by_store(val):
    clear_output()
    display(dropdown)
    pick_data = order_data.loc[(order_data["store_name"]==val["new"]) & (order_data["status"].isin([1,2]))]
    display(pick_data.head())

store_list = m_store["store_name"].tolist()

dropdown = Dropdown(options=store_list)
dropdown.observe(order_by_store, names="value")
display(dropdown)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib

def graph_by_store(val):
    clear_output()
    display(dropdown2)
    pick_data = order_data.loc[(order_data["store_name"]==val["new"]) & (order_data["status"].isin([1,2]))]
    temp =pick_data[["order_accept_date", "total_amount"]].copy()
    temp.loc[:,"order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
    temp.set_index("order_accept_date", inplace=True)
    temp.resample("D").sum().plot()

dropdown2 = Dropdown(options=store_list)
dropdown2.observe(graph_by_store, names="value")
display(dropdown2)


### ノック２２：複数店舗の詳細を可視化できるようにしてみよう

In [ ]:
from ipywidgets import SelectMultiple

def order_by_multi(val):
    clear_output()
    display(select)
    pick_data = order_data.loc[(order_data["store_name"].isin(val["new"])) & (order_data["status"].isin([1,2]))]
    display(pick_data.head())

select = SelectMultiple(options=store_list)
select.observe (order_by_multi, names="value")
display(select)

### ノック２３：スライドバーを用いてオーダー件数を調べてみよう

### ノック２４：トグルボタンで地域データを抽出しよう

### ノック２５：日付を指定してデータを抽出してみよう

### ノック２６：ストーリーを考えてデータを構築しよう

### ノック２７：キャンセルの理由を分析してみよう

### ノック２８：仮説を検証してみよう

### ノック２９：ストーリーをもとにパーツやデータを組み合わせてダッシュボードを作ろう

### ノック３０：ダッシュボードを改善しよう